# This script is used to calculate NPMI and get gene-phenotype matrix

In [41]:
import glob
import json
import numpy as np
import math
import os
import ast
import pandas as pd
import csv
import xlrd

In [42]:
#define dir

# input file dir
json_path = '.\\Extraced_results'              # the output file of step1
np_dir = '.\\Sum_all\\n_p.txt'                 # the output file of step1
ng_dir = '.\\Sum_all\\n_g.txt'                 # the output file of step1
In_Summary_dir='.\\Sum_all\\In_Summary.txt'    # the output file of step1
sfari_gene_dir='.\\Genotype_list\\SFARI-Gene_genes_12-11-2020release_12-19-2020export.xlsx'# the SFARI genes file dir

# output file dir
NPMI_result_dir='.\\NPMI_file\\'               # folder of NPMI file 

In [ ]:
# other definations
genes_outlier = ['BDNF']# define gene outlier
jsons = glob.glob("{}\\*.json".format(json_path)) # get the dir of every file in .\\Extraced_results
NPMI_dir=NPMI_result_dir+'NPMI.json' # raw NPMI json file
NPMI_csv_dir=NPMI_result_dir+'NPMI.csv'# raw NPMI csv file
NPMI_above_zero_csv_dir=NPMI_result_dir+'NPMI_above_zero.csv'# pairs with NPMI above 0, csv file
graph_matrix_dir = NPMI_result_dir+'graph_matrix_01_NPMIabove0.csv'# matrix, NPMI>0 -> 1,NPMI<=0 -> 0

In [43]:
# delete old output file
def delete_old_file():
    if os.path.exists(NPMI_dir):
        os.remove(NPMI_dir)
    if os.path.exists(NPMI_csv_dir):
        os.remove(NPMI_csv_dir)
    if os.path.exists(graph_matrix_dir):
        os.remove(graph_matrix_dir)
    if os.path.exists(NPMI_above_zero_csv_dir):
        os.remove(NPMI_above_zero_csv_dir)  

In [44]:
# this cell is used to define functions that can read files

# read file in .\\Sum_all
def read_file():
    #read n_p
    with open(np_dir, 'r', encoding='utf-8-sig') as o:  
        phenotype_dict =  ast.literal_eval(o.read())
    
            
    #read n_g
    with open(ng_dir, 'r', encoding='utf-8-sig') as o:  
        gene_dict =  ast.literal_eval(o.read())
            
    #read n_tot
    with open(In_Summary_dir, 'r', encoding='utf-8-sig') as o:  
        f=o.read()
        h = f.find('N_tot = ')
        e = f.find('Unique gene list from all papers:')
        n_tot = int(f[h+8:e-2])
    return phenotype_dict, gene_dict, n_tot

# read sfari file in .\\Genotype_list
def read_sfari(sfari_gene_dir):
    wb = xlrd.open_workbook(sfari_gene_dir)
    sh = wb.sheet_by_name('Sheet2')
    sfari_gene_name = sh.col_values(1)[1:]
    sfari_gene_class = sh.col_values(2)[1:]
    sfari_gene_dict = dict(zip(sfari_gene_name,sfari_gene_class))  
    return sfari_gene_dict

In [45]:
# this cell is used to define functions that modify the gene_dict

# delete one gene from gene_dict
def delete_gene_from_dict(gene_delete_list,gene_dict):
    for gene in gene_delete_list:
        if gene in gene_dict.keys():
            del gene_dict[gene]
    return gene_dict

# delete 2 words genes in gene_dict 
def delete_2word_gene_from_dict(gene_dict):
    for gene in list(gene_dict.keys()):
        if len(gene)<=2:
            del gene_dict[gene]
    return gene_dict

In [46]:
# get gene_list and phenotype list from gene_dict and phenotype_dict
def get_list(gene_dict,phenotype_dict):
    gene_list=[]
    for key in gene_dict.keys():
        gene_list.append(key)
    phenotype_list=[]
    for key in phenotype_dict.keys():
        phenotype_list.append(key)
    return gene_list,phenotype_list

In [47]:
# generate matrix that shows the relationship of gene and phenotype. 
# row is gene and column is phenotype, the crosspoints are the co-occurance time of gene and phenotype

def generate_matrix (gene_list,phenotype_list,gene_dict,phenotype_dict):
    graph_matrix = np.zeros((len(gene_list),len(phenotype_list)))
    for index,item in enumerate(jsons) :
        if index%100 ==0:
            print(index)# show the process of calculating NPMI
        with open(item, 'r', encoding='utf-8-sig') as f:
            json_data = json.load(f)
            for key in json_data['Sentences']:
                json_phenotype = json_data['Sentences'][key]['Normolized phenotype']
                json_gene = json_data['Sentences'][key]['Gene']
                for phenotype in json_phenotype:
                    phenotype_in_dict = [x for i,x in enumerate(phenotype_list) if x.find(str(phenotype))!=-1][0]

                    for gene in json_gene:
                        if gene in gene_list:
                            graph_matrix[gene_list.index(gene)][phenotype_list.index(phenotype_in_dict)] += 1
    return graph_matrix

In [48]:
# calculate NPMI and output NPMI raw data file (json file and csv file)
def NPMI_calculate_and_output_NPMI_file(gene_list,phenotype_list,gene_dict,phenotype_dict,graph_matrix,n_tot,sfari_gene_dict):
# calculate NPMI and output NPMI raw data json file
    data_of_all_pairs=[]                
    with open(NPMI_dir, 'w', encoding='utf-8-sig') as f:    
        count_pair = 0           
        for phenotype_index, phenotype in enumerate(phenotype_list):
            for gene_index,gene in enumerate(gene_list):
                n_g = gene_dict[gene]
                n_p = phenotype_dict[phenotype]
                n_gp = graph_matrix[gene_index][phenotype_index]
                if n_gp!= 0:
                    NPMI = math.log((n_gp*n_tot)/(n_g*n_p))/(-math.log(n_gp/n_tot))# calculate NPMI
                    
                    if gene in sfari_gene_dict.keys():
                        gene_class = sfari_gene_dict[gene]
                    else:
                        gene_class = 'NA'
                    data_of_one_pair = {'gene':gene, 'phenotype':phenotype,'NPMI': NPMI,'gene_sfari_class':gene_class, 'n_g':n_g, 'n_p':n_p, 'n_gp':n_gp}
                    data_of_all_pairs.append(data_of_one_pair)
                    count_pair += 1
                    #f.write(tplt.format(gene, phenotype, NPMI,n_g,n_p,n_gp))
        data_of_all_pairs.sort(key=lambda x: x["NPMI"])
        json.dump(data_of_all_pairs , f , sort_keys=False, indent=4, separators=(',', ': '))
    all_pair_NPMI_result = data_of_all_pairs
    

# use a dict to store NPMI data sorted by gene
    pairs_sort_by_gene_dict = {}
    for gene in gene_list:
        same_gene_pair_data=[]
        for item in data_of_all_pairs:
            if item['gene']==gene:
                same_gene_pair_data.append(item)
        same_gene_pair_data.sort(key=lambda x: -x["NPMI"])
        pairs_sort_by_gene_dict[gene] = same_gene_pair_data


# output NPMI raw data csv file
    with open(NPMI_csv_dir, 'w', encoding='utf-8-sig',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["gene", "phenotype","gene_sfari_class", "NPMI","n_g","n_p","n_gp"])
        for gene in gene_list:
            one_gene_json_list = pairs_sort_by_gene_dict[gene] 
            for item in one_gene_json_list:
                writer.writerows([(item['gene'], item['phenotype'],item['gene_sfari_class'], item['NPMI'],item['n_g'],item['n_p'],item['n_gp'])])
    return all_pair_NPMI_result
                    

In [49]:
# output pairs that NPMI>0, csv file
def output_NPMI_above_zero_csv(NPMI_above_zero_csv_dir,all_pair_NPMI_result):
    all_pair_NPMI_result.sort(key=lambda x: x["NPMI"])
    with open(NPMI_above_zero_csv_dir, 'w', encoding='utf-8-sig',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["gene", "phenotype","gene_sfari_class", "NPMI","n_g","n_p","n_gp"])
        for item in all_pair_NPMI_result:
            if item['NPMI']>0:
                writer.writerows([(item['gene'], item['phenotype'],item['gene_sfari_class'], item['NPMI'],item['n_g'],item['n_p'],item['n_gp'])])


In [50]:
# modify the graph_matrix
# row is gene and column is phenotype, 
# crosspoints: if NPMI of this pair is above 0, then the crosspoint would be 1. else, the crosspoint would be 0
# Also, delete the row and column that only have 0. Also delete gene and phenotype in gene_list and phenotype_list

def modify_to_01_NPMIabove0(graph_matrix,NPMI_result,gene_list,phenotype_list):
    graph_matrix = np.ones((len(gene_list),len(phenotype_list)))*-1#row, column
    for item in NPMI_result:
        gene = item['gene']
        phenotype = item['phenotype']
        NPMI = item['NPMI']
        graph_matrix[gene_list.index(gene)][phenotype_list.index(phenotype)] =NPMI
        
    graph_matrix[graph_matrix>0]=1# NPMI>0 -> 1
    graph_matrix[graph_matrix<=0]=0#,NPMI<=0 -> 0

    all_zero_row = list(np.where(~graph_matrix.any(axis=1))[0]) #find the all 0 row
    graph_matrix = np.delete(graph_matrix,all_zero_row,axis=0)
    for i in reversed(all_zero_row):
        del gene_list[i]
    
    all_zero_col = list(np.where(~graph_matrix.any(axis=0))[0]) #find the all 0 col
    graph_matrix = np.delete(graph_matrix,all_zero_col,axis=1)
    for i in reversed(all_zero_col):
        del phenotype_list[i]
    
    return graph_matrix, gene_list, phenotype_list

In [51]:
# output the modified matrix
def out_put_graph(graph_matrix_dic,graph_matrix,gene_list,phenotype_list):
    graph_matrix = pd.DataFrame(graph_matrix, index=gene_list, columns=phenotype_list)
    graph_matrix.to_csv(graph_matrix_dic,sep=',',index=True,header=True)
        

In [52]:
# main function

if __name__ == '__main__':
    # read file
    delete_old_file()
    phenotype_dict, gene_dict, n_tot = read_file()
    sfari_gene_dict = read_sfari(sfari_gene_dir)
        
    # delete outlier
    gene_dict = delete_gene_from_dict(genes_outlier,gene_dict)

    # delete 2 words genes
    gene_dict = delete_2word_gene_from_dict(gene_dict)
    
    # get list
    gene_list,phenotype_list = get_list(gene_dict,phenotype_dict)
    
    graph_matrix= generate_matrix(gene_list,phenotype_list,gene_dict,phenotype_dict)

    all_pair_NPMI_result = NPMI_calculate_and_output_NPMI_file(gene_list,phenotype_list,gene_dict,phenotype_dict,graph_matrix,n_tot,sfari_gene_dict) 
    
    output_NPMI_above_zero_csv(NPMI_above_zero_csv_dir,all_pair_NPMI_result)

    graph_matrix, gene_list, phenotype_list = modify_to_01_NPMIabove0(graph_matrix,all_pair_NPMI_result,gene_list,phenotype_list)
    out_put_graph(graph_matrix_dir,graph_matrix,gene_list,phenotype_list)



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
